In [ ]:
Строим логистическую регрессию - угадываем доход человека

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv( 'adult.csv' , na_values='?' ) #не заполненные значения в виде ? сразу преобразуем в NA)
data.head(10)
        

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,NaN,227026,HS-grad,9,Never-married,NaN,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [5]:
df=data[['age', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'hours-per-week',
         'capital-gain', 'capital-loss', 'native-country', 'income']] #Оставляем колонки по которым будем делать предсказание 
# и результирующий класс
df=df.dropna() #удаляем NA, чтобы убрать их влияние на предсказание

In [6]:
df.head(10)

,age,workclass,education,marital-status,occupation,relationship,race,gender,hours-per-week,capital-gain,capital-loss,native-country,income
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,40,0,0,United-States,<=50K
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,50,0,0,United-States,<=50K
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,40,0,0,United-States,>50K
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,40,7688,0,United-States,>50K
5,34,Private,10th,Never-married,Other-service,Not-in-family,White,Male,30,0,0,United-States,<=50K
7,63,Self-emp-not-inc,Prof-school,Married-civ-spouse,Prof-specialty,Husband,White,Male,32,3103,0,United-States,>50K
8,24,Private,Some-college,Never-married,Other-service,Unmarried,White,Female,40,0,0,United-States,<=50K
9,55,Private,7th-8th,Married-civ-spouse,Craft-repair,Husband,White,Male,10,0,0,United-States,<=50K
10,65,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,6418,0,United-States,>50K
11,36,Federal-gov,Bachelors,Married-civ-spouse,Adm-clerical,Husband,White,Male,40,0,0,United-States,<=50K


In [7]:
from sklearn.linear_model import LogisticRegression

# импортируем метод для автоматической разбивки на обучающую и тестовую выборки

from sklearn.model_selection import train_test_split

In [8]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = df[ [ 'age', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'hours-per-week',
         'capital-gain', 'capital-loss', 'native-country', 'income']]
# столбецы с  категориальной переменной
# переведем  в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies( selectedColumns, columns = [ 'workclass', 'education','marital-status', 'occupation',
                                                'relationship', 'race', 'gender', 'native-country' ] )

# столбец 'income' является целевой переменной, удаляем его из X
del X['income']
X.head()

,age,hours-per-week,capital-gain,capital-loss,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,40,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,50,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,40,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,40,7688,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,30,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [9]:

# целевая переменная (столбец 'income') снова является категориальной
# переведем значения столбца в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [10]:
le.fit( df['income'] )

LabelEncoder()

In [11]:
# записываем в переменную y преобразованный столбец sex

y = pd.Series(data = le.transform( df['income'] ) )
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [12]:
model = LogisticRegression()

In [13]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.3, random_state = 0)
model.fit( X_train, y_train )
predictions = model.predict_proba( X_test )

C:\Users\DDD\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
predictions[:5]

array([[0.92797798, 0.07202202],
       [0.97738042, 0.02261958],
       [0.78542786, 0.21457214],
       [0.81446463, 0.18553537],
       [0.98935023, 0.01064977]])

In [14]:
model.score(X_test, y_test)

0.8320925775779465

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [16]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [26]:
clf.score(X_test, y_test)

0.8401267782118376

In [17]:
svc = SVC(gamma='auto')
svc.fit(X_train, y_train)

SVC(gamma='auto')

In [18]:
svc.score(X_test, y_test)

0.8541313481241247